In [1]:
import pandas as pd
import numpy as np
import re
import difflib
pd.set_option('display.max_rows', 2500)

- master df (ΔΗΜΟΙ) που περιέχουν και τις έκτακτες χρηματοδοτήσεις.
- Δαπάνες από τη Διαύγεια (AGG_ΔΗΜΟΙ)
- κωδικοί (uid) από τη Διαύγεια + same_us ([F} Diavgeia_Mun_Lookup)
- χρηματοδοτήσεις δήμων για πυροπροστασία από κεντρικούς αυτοτελείς πόρους (KAP_2018_2022) - Υπουργείο Εσωτερικών
- πυρκαγιές ανά έτος - δήμο (fires_grouped_mun_year)

In [2]:
uid = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSFvt2TO_6Xbv4d2aUxI8n9P6UcFAcl7n5B09CAM1isk5XwLgERhEAEAaUCDnMH0v9KpqXCasZIFxns/pub?gid=1029651505&single=true&output=csv'
master = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSFvt2TO_6Xbv4d2aUxI8n9P6UcFAcl7n5B09CAM1isk5XwLgERhEAEAaUCDnMH0v9KpqXCasZIFxns/pub?gid=0&single=true&output=csv'
ypes = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSFvt2TO_6Xbv4d2aUxI8n9P6UcFAcl7n5B09CAM1isk5XwLgERhEAEAaUCDnMH0v9KpqXCasZIFxns/pub?gid=223369667&single=true&output=csv'
payments = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSFvt2TO_6Xbv4d2aUxI8n9P6UcFAcl7n5B09CAM1isk5XwLgERhEAEAaUCDnMH0v9KpqXCasZIFxns/pub?gid=2046776889&single=true&output=csv'
fires = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSFvt2TO_6Xbv4d2aUxI8n9P6UcFAcl7n5B09CAM1isk5XwLgERhEAEAaUCDnMH0v9KpqXCasZIFxns/pub?gid=417193089&single=true&output=csv'

In [3]:
# corrections of municipalities names in the ypes data (normalized with DIAVGEIA)
# Το key είναι όπως αναφέρεται στο ΔΗΜΟΙ  value όπως αναφέρεται στο spreadsheet KAP 2018_2022.
municipalities = {
    'ΓΕΩΡΓΙΟΥ ΚΑΡΑΙΣΚΑΚΗ':'ΓΕΩΡΓΙΟΥ ΚΑΡΑΪΣΚΑΚΗ',
    'ΔΥΤΙΚΗΣ ΑΧΑΙΑΣ':'ΔΥΤΙΚΗΣ ΑΧΑΪΑΣ',
    'ΒΟΙΟΥ':'ΒΟΪΟΥ',
    'ΗΛΙΟΥΠΟΛΗΣ':'ΗΛΙΟΥΠΟΛΕΩΣ',
    'ΘΕΡΜΑΙΚΟΥ':'ΘΕΡΜΑΪΚΟΥ',
    'ΚΑΛΑΜΠΑΚΑΣ':'ΜΕΤΕΩΡΩΝ',
    'ΚΑΣΟΥ':'ΗΡΩΙΚΗΣ ΝΗΣΟΥ ΚΑΣΟΥ',
    'ΚΑΣΟΥ':'ΗΡΩΪΚΗΣ ΝΗΣΟΥ ΚΑΣΟΥ',
    'ΚΑΣΟΥ':'ΗΡΩΙΚΗΣ ΝΗΣΟΥ ΚΑΣΟΥ',
    'ΚΑΣΟΥ':'ΗΡΩΪΚΗΣ ΝΗΣΟΥ ΚΑΣΟΥ',
    'ΚΕΝΤΡΙΚΗΣ ΚΕΡΚΥΡΑΣ ΚΑΙ ΔΙΑΠΟΝΤΙΩΝ ΝΗΣΩΝ':'ΚΕΝΤΡΙΚΗΣ ΚΕΡΚΥΡΑΣ &',
    'ΚΕΝΤΡΙΚΗΣ ΚΕΡΚΥΡΑΣ ΚΑΙ ΔΙΑΠΟΝΤΙΩΝ ΝΗΣΩΝ':'ΚΕΝΤΡΙΚΗΣ ΚΕΡΚΥΡΑΣ &',
    'ΚΕΝΤΡΙΚΗΣ ΚΕΡΚΥΡΑΣ ΚΑΙ ΔΙΑΠΟΝΤΙΩΝ ΝΗΣΩΝ':'ΚΕΝΤΡΙΚΗΣ ΚΕΡΚΥΡΑΣ & ΔΙΑΠΟΝΤΙΩΝ ΝΗΣΩΝ',
    'ΜΕΤΑΜΟΡΦΩΣΗΣ':'ΜΕΤΑΜΟΡΦΩΣΕΩΣ',
    'ΜΩΛΟΥ-ΑΓ. ΚΩΝΣΤΑΝΤΙΝΟΥ':'ΚΑΜΕΝΩΝ ΒΟΥΡΛΩΝ',
    'ΜΩΛΟΥ-ΑΓ. ΚΩΝΣΤΑΝΤΙΝΟΥ':'ΚΑΜΕΝΩΝ ΒΟΥΡΛΩΝ',
    'ΜΩΛΟΥ-ΑΓ. ΚΩΝΣΤΑΝΤΙΝΟΥ':'ΜΩΛΟΥ-ΑΓΙΟΥ ΚΩΝΣΤΑΝΤΙΝΟΥ',
    'ΝΑΞΟΥ & ΜΙΚΡΩΝ ΚΥΚΛΑΔΩΝ':'ΝΑΞΟΥ ΚΑΙ ΜΙΚΡΩΝ ΚΥΚΛΑΔΩΝ',
    'ΝΑΟΥΣΑΣ':'ΗΡΩΙΚΗΣ ΠΟΛΕΩΣ ΝΑΟΥΣΑΣ',
    'ΝΑΟΥΣΑΣ':'ΗΡΩΙΚΗΣ ΠΟΛΕΩΣ ΝΑΟΥΣΑΣ',
    'ΝΑΟΥΣΑΣ':'ΗΡΩΙΚΗΣ ΠΟΛΗΣ ΝΑΟΥΣΑΣ',
    'ΝΙΚΑΙΑΣ-ΑΓΙΟΥ Ι. ΡΕΝΤΗ':'ΝΙΚΑΙΑΣ-ΑΓΙΟΥ ΙΩΑΝΝΗ ΡΕΝΤΗ',
    'ΠΕΤΡΟΥΠΟΛΗΣ':'ΠΕΤΡΟΥΠΟΛΕΩΣ',
    'ΧΑΙΔΑΡΙΟΥ':'ΧΑΪΔΑΡΙΟΥ',
    'ΨΑΡΩΝ':'ΗΡΩΙΚΗΣ ΝΗΣΟΥ ΨΑΡΩΝ',
    'ΨΑΡΩΝ':'ΗΡΩΪΚΗ ΝΗΣΟΣ ΨΑΡΩΝ',
    'ΛΕΥΚΑΔΑΣ':'ΛΕΥΚΑΔΟΣ',
    'ΦΙΛΑΔΕΛΦΕΙΑΣ-ΧΑΛΚΗΔΟΝΟΣ':' ΝΕΑΣ  ΦΙΛΑΔΕΛΦΕΙΑΣ-ΝΕΑΣ ΧΑΛΚΗΔΟΝΑΣ',
    'ΛΟΥΤΡΑΚΙΟΥ-ΠΕΡΑΧΩΡΑΣ-ΑΓΙΩΝ ΘΕΟΔΩΡΩΝ':'ΛΟΥΤΡΑΚΙΟΥ-ΑΓΙΩΝ ΘΕΟΔΩΡΩΝ',
    'ΚΕΦΑΛΟΝΙΑΣ':'ΚΕΦΑΛΛΟΝΙΑΣ',
    'ΦΙΛΟΘΕΗΣ-ΨΥΧΙΚΟΥ':'ΦΙΛΟΘΕΗΣ -ΨΥΧΙΚΟΥ',
    'ΑΡΓΟΥΣ ΟΡΕΣΤΙΚΟΥ':'ΑΡΓΟΣ ΟΡΕΣΤΙΚΟΥ',
    'ΤΡΟΙΖΗΝΙΑΣ-ΜΕΘΑΝΩΝ':'ΤΡΟΙΖΗΝΙΑΣ',
    'ΤΡΙΦΥΛΙΑΣ':'ΤΡΙΦΥΛΛΙΑΣ',
    'ΝΙΚΑΙΑΣ-ΑΓΙΟΥ Ι. ΡΕΝΤΗ':'ΝΙΚΑΙΑΣ-ΑΓΙΟΥ ΙΩΑΝΝΗ ΡΕΝΤΗ'   
}

# Functions

In [4]:
def strip_rows(data):
    return data.replace(r'(ΔΗΜΟΣ?\s+)','', regex=True)\
                .replace(r'( - )','-', regex=True)\
                .replace(r'(\s{2,})',' ', regex=True)\
                .replace(r'(^\s*)','', regex=True)\
                .replace(r'(^A)','Α', regex=True)


In [5]:
def normalize_municipality_names(data, date, new_name, name, column_name):
    '''
    data = to dataframe
    date = το προηγούμενο έτος από το οποίο θέλουμε να ξεκινήσει η αλλαγή
    new_name = το νέο value
    name = το παλιό value
    column_name = το όνομα της στήλης
    '''
    indexes = data[(data['year']>date) & (data[column_name]==name)].index.tolist()
    for idx in indexes:
        data.at[idx, column_name]=new_name
    return data

# Διαύγεια uid

In [6]:
uid_df = pd.read_csv(uid, skipinitialspace = True, na_filter=False)

In [7]:
uid_df = uid_df[['uid','same uid as','label']].copy()

In [8]:
uid_df.rename(columns={'label':'municipality'},inplace=True)

In [9]:
uid_df= uid_df.apply(strip_rows)

In [10]:
uid_df.head(1)

uid same uid as municipality
0  6001                  ΑΒΔΗΡΩΝ

In [11]:
uid_df[uid_df['municipality'].str.contains('ΑΛΙΑΡΤ', na=False)]

uid same uid as       municipality
19   6020                       ΑΛΙΑΡΤΟΥ
341  6020              ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ

# Tab Δήμοι από google spreadsheet

In [12]:
master_df = pd.read_csv(master, skipinitialspace = True, na_filter=False)

In [13]:
master_df = master_df[['Έτος', 'Δήμος', 'cleaned dimos', 'Περιφέρεια', 'Δασαρχείο',
       'Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας', 'Συμπληρωματική χρηματοδότηση ΥΠΕΣ',
       'Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας', 'Χρηματοδότηση ΕΣΠΑ',
       'Χρηματοδότηση Αντώνης Τρίτσης', 'Άλλη πηγή χρηματοδότησης ονομαστικά',
       'Δαπάνη για αντιπυρρική προστασία - Ισολογισμός']].copy()

In [14]:
master_df['cleaned dimos'].replace('ΔΗΜΟ ΑΡΓΟΥΣ ΟΡΕΣΤΙΚΟΥ','ΑΡΓΟΥΣ ΟΡΕΣΤΙΚΟΥ', inplace=True)

In [15]:
master_df = master_df.apply(strip_rows)

In [16]:
master_df.shape


(1705, 12)

In [17]:
master_df.rename(columns={'Έτος':'year', 'cleaned dimos':'municipality'}, inplace=True)

In [18]:
master_df

year                                              Δήμος  \
0     2022                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
1     2022                                            ΑΧΑΡΝΩΝ   
2     2022                                           ΓΛΥΦΑΔΑΣ   
3     2022                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
4     2022                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
5     2022                                          ΜΥΤΙΛΗΝΗΣ   
6     2022                                           ΠΕΝΤΕΛΗΣ   
7     2022                                              ΡΟΔΟΥ   
8     2022                                           ΣΟΥΦΛΙΟΥ   
9     2022                                             ΩΡΩΠΟΥ   
10    2022                                           ΑΛΜΩΠΙΑΣ   
11    2022                                           ΑΝΩΓΕΙΩΝ   
12    2022                                            ΑΒΔΗΡΩΝ   
13    2022                                        ΑΓΑΘΟΝΗΣΙΟΥ   
14    2022                                              ΑΓΙΑΣ   
15    2022                                     ΑΓΙΑΣ ΒΑΡΒΑΡΑΣ   
16    2022                                   ΑΓΙΑΣ ΠΑΡΑΣΚΕΥΗΣ   
17    2022                                    ΑΓΙΟΥ ΒΑΣΙΛΕΙΟΥ   
18    2022                                    ΑΓΙΟΥ ΔΗΜΗΤΡΙΟΥ   
19    2022                                   ΑΓΙΟΥ ΕΥΣΤΡΑΤΙΟΥ   
20    2022                          ΑΓΙΩΝ ΑΝΑΡΓΥΡΩΝ-ΚΑΜΑΤΕΡΟΥ   
21    2022                                         ΑΓΚΙΣΤΡΙΟΥ   
22    2022                                            ΑΓΡΑΦΩΝ   
23    2022                                           ΑΓΡΙΝΙΟΥ   
24    2022                                           ΑΘΗΝΑΙΩΝ   
25    2022                                            ΑΙΓΑΛΕΩ   
26    2022                                         ΑΙΓΙΑΛΕΙΑΣ   
27    2022                                            ΑΙΓΙΝΑΣ   
28    2022                                    ΑΚΤΙΟΥ-ΒΟΝΙΤΣΑΣ   
29    2022                                       ΑΛΕΞΑΝΔΡΕΙΑΣ   
30    2022                                    ΑΛΕΞΑΝΔΡΟΥΠΟΛΗΣ   
31    2022                                  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   
32    2022                                             ΑΛΙΜΟΥ   
33    2022                                            ΑΛΜΥΡΟΥ   
34    2022                                          ΑΛΟΝΝΗΣΟΥ   
35    2022                                            ΑΜΑΡΙΟΥ   
36    2022                                         ΑΜΑΡΟΥΣΙΟΥ   
37    2022                                            ΑΜΟΡΓΟΥ   
38    2022                              ΑΜΠΕΛΟΚΗΠΩΝ-ΜΕΝΕΜΕΝΗΣ   
39    2022                                          ΑΜΥΝΤΑΙΟΥ   
40    2022                                ΑΜΦΙΚΛΕΙΑΣ-ΕΛΑΤΕΙΑΣ   
41    2022                                         ΑΜΦΙΛΟΧΙΑΣ   
42    2022                                          ΑΜΦΙΠΟΛΗΣ   
43    2022                                   ΑΝΑΤΟΛΙΚΗΣ ΜΑΝΗΣ   
44    2022                                             ΑΝΑΦΗΣ   
45    2022                                ΑΝΔΡΑΒΙΔΑΣ-ΚΥΛΛΗΝΗΣ   
46    2022                             ΑΝΔΡΙΤΣΑΙΝΑΣ-ΚΡΕΣΤΕΝΩΝ   
47    2022                                             ΑΝΔΡΟΥ   
48    2022                                          ΑΝΤΙΠΑΡΟΥ   
49    2022                                         ΑΠΟΚΟΡΩΝΟΥ   
50    2022                                           ΑΡΓΙΘΕΑΣ   
51    2022                                     ΑΡΓΟΥΣ-ΜΥΚΗΝΩΝ   
52    2022                                         ΑΡΙΣΤΟΤΕΛΗ   
53    2022                                           ΑΡΡΙΑΝΩΝ   
54    2022                                            ΑΡΤΑΙΩΝ   
55    2022                                ΑΡΧΑΝΩΝ-ΑΣΤΕΡΟΥΣΙΩΝ   
56    2022                                        ΑΣΠΡΟΠΥΡΓΟΥ   
57    2022                                        ΑΣΤΥΠΑΛΑΙΑΣ   
58    2022                          ΒΑΡΗΣ-ΒΟΥΛΑΣ-ΒΟΥΛΙΑΓΜΕΝΗΣ   
59    2022                                        ΒΕΛΟΥ-ΒΟΧΑΣ   
60    2022                               

# Επιδοτήσεις δήμων από Κεντρικούς Αυτοτελής Πόρους

In [19]:
ypes_df = pd.read_csv(ypes, skipinitialspace = True, na_filter=False)

In [20]:
ypes_df = ypes_df.apply(strip_rows)

In [21]:
ypes_df = ypes_df[['year','municipality','amount']].copy()

In [22]:
ypes_df.head(1)

year municipality  amount
0  2022     ΑΓΡΙΝΙΟΥ  144000

In [23]:
ypes_df.shape

(1616, 3)

In [24]:
# correct ypes municipalities data
ypes_df['municipality'].replace(municipalities.values(), municipalities.keys(), inplace=True)

In [25]:
ypes_df


year                             municipality  amount
0     2022                                 ΑΓΡΙΝΙΟΥ  144000
1     2022                          ΑΚΤΙΟΥ-ΒΟΝΙΤΣΑΣ   44000
2     2022                               ΑΜΦΙΛΟΧΙΑΣ   61000
3     2022                                   ΘΕΡΜΟΥ   20000
4     2022                  ΙΕΡΑΣ ΠΟΛΗΣ ΜΕΣΟΛΟΓΓΙΟΥ   48900
5     2022                               ΝΑΥΠΑΚΤΙΑΣ   78000
6     2022                                ΞΗΡΟΜΕΡΟΥ   43000
7     2022                           ΑΡΓΟΥΣ-ΜΥΚΗΝΩΝ  145000
8     2022                                ΕΠΙΔΑΥΡΟΥ   65000
9     2022                               ΕΡΜΙΟΝΙΔΑΣ   52000
10    2022                                ΝΑΥΠΛΙΕΩΝ   79000
11    2022                        ΒΟΡΕΙΑΣ ΚΥΝΟΥΡΙΑΣ   32800
12    2022                                ΓΟΡΤΥΝΙΑΣ  108000
13    2022                              ΜΕΓΑΛΟΠΟΛΗΣ   66700
14    2022                         ΝΟΤΙΑΣ ΚΥΝΟΥΡΙΑΣ   53000
15    2022                                 ΤΡΙΠΟΛΗΣ  149000
16    2022                                  ΑΡΤΑΙΩΝ   51000
17    2022                      ΓΕΩΡΓΙΟΥ ΚΑΡΑΙΣΚΑΚΗ   20600
18    2022                     ΚΕΝΤΡΙΚΩΝ ΤΖΟΥΜΕΡΚΩΝ   28400
19    2022                          ΝΙΚΟΛΑΟΥ ΣΚΟΥΦΑ   20600
20    2022                           ΑΓΙΑΣ ΒΑΡΒΑΡΑΣ   27000
21    2022                         ΑΓΙΑΣ ΠΑΡΑΣΚΕΥΗΣ   24800
22    2022                          ΑΓΙΟΥ ΔΗΜΗΤΡΙΟΥ   14600
23    2022                ΑΓΙΩΝ ΑΝΑΡΓΥΡΩΝ-ΚΑΜΑΤΕΡΟΥ   24800
24    2022                               ΑΓΚΙΣΤΡΙΟΥ   15000
25    2022                                 ΑΘΗΝΑΙΩΝ  118000
26    2022                                  ΑΙΓΑΛΕΩ   22500
27    2022                                  ΑΙΓΙΝΑΣ   24700
28    2022                               ΑΜΑΡΟΥΣΙΟΥ   18000
29    2022                              ΑΣΠΡΟΠΥΡΓΟΥ   27000
30    2022                                  ΑΧΑΡΝΩΝ   63000
31    2022                ΒΑΡΗΣ-ΒΟΥΛΑΣ-ΒΟΥΛΙΑΓΜΕΝΗΣ   52600
32    2022                               ΒΡΙΛΗΣΣΙΩΝ   22600
33    2022                                  ΒΥΡΩΝΟΣ   27000
34    2022                                ΓΑΛΑΤΣΙΟΥ   28800
35    2022                                 ΓΛΥΦΑΔΑΣ   22000
36    2022                           ΔΑΦΝΗΣ-ΥΜΗΤΤΟΥ   37600
37    2022                                 ΔΙΟΝΥΣΟΥ  131700
38    2022                                ΕΛΕΥΣΙΝΑΣ   36400
39    2022                   ΕΛΛΗΝΙΚΟΥ-ΑΡΓΥΡΟΥΠΟΛΗΣ   26000
40    2022                                 ΖΩΓΡΑΦΟΥ   24000
41    2022                               ΗΛΙΟΥΠΟΛΗΣ   27000
42    2022                                ΗΡΑΚΛΕΙΟΥ   19500
43    2022                                    ΙΛΙΟΥ   24600
44    2022                              ΚΑΙΣΑΡΙΑΝΗΣ   32000
45    2022                  ΚΕΡΑΤΣΙΝΙΟΥ-ΔΡΑΠΕΤΣΩΝΑΣ   20000
46    2022                                 ΚΗΦΙΣΙΑΣ   79500
47    2022                               ΚΟΡΥΔΑΛΛΟΥ   14500
48    2022                                  ΚΡΩΠΙΑΣ   61600
49    2022                                  ΚΥΘΗΡΩΝ   59700
50    2022                              ΛΑΥΡΕΩΤΙΚΗΣ   89400
51    2022                        ΛΥΚΟΒΡΥΣΗΣ-ΠΕΥΚΗΣ   29500
52    2022                        ΜΑΝΔΡΑΣ-ΕΙΔΥΛΛΙΑΣ  101200
53    2022                                ΜΑΡΑΘΩΝΟΣ   92800
54    2022                    ΜΑΡΚΟΠΟΥΛΟΥ ΜΕΣΟΓΑΙΑΣ   18000
55    2022                                 ΜΕΓΑΡΕΩΝ   50800
56    2022                             ΜΕΤΑΜΟΡΦΩΣΗΣ   15000
57    2022                              ΝΕΑΣ ΙΩΝΙΑΣ   20000
58    2022        ΝΕΑΣ ΦΙΛΑΔΕΛΦΕΙΑΣ-ΝΕΑΣ ΧΑΛΚΗΔΟΝΑΣ   36400
59    2022                   ΝΙΚΑΙΑΣ-ΑΓΙΟΥ Ι. ΡΕΝΤΗ   23600
60    2022                                 ΠΑΙΑΝΙΑΣ   40000
61    2022                                 ΠΑΛΛΗΝΗΣ   48000
62    2022                         ΠΑΠΑΓΟΥ-ΧΟΛΑΡΓΟΥ   38400
63    2022                                 ΠΕΝΤΕΛΗΣ   75000
64    2022                                ΠΕΡΑΜΑΤΟΣ   18000
65    2022                              ΠΕΡΙΣΤ

# Σύνδεση κεντρικού data (Δήμοι) με ΥΠΕΣ χρηματοδοτήσεις

In [26]:
# merge master with ypes
df = pd.merge(master_df, ypes_df, left_on=['year','municipality'], right_on=['year','municipality'], how='outer')

In [27]:
df

year                                              Δήμος  \
0     2022                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
1     2022                                            ΑΧΑΡΝΩΝ   
2     2022                                           ΓΛΥΦΑΔΑΣ   
3     2022                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
4     2022                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
5     2022                                          ΜΥΤΙΛΗΝΗΣ   
6     2022                                          ΜΥΤΙΛΗΝΗΣ   
7     2022                                           ΠΕΝΤΕΛΗΣ   
8     2022                                              ΡΟΔΟΥ   
9     2022                                           ΣΟΥΦΛΙΟΥ   
10    2022                                             ΩΡΩΠΟΥ   
11    2022                                           ΑΛΜΩΠΙΑΣ   
12    2022                                           ΑΝΩΓΕΙΩΝ   
13    2022                                            ΑΒΔΗΡΩΝ   
14    2022                                        ΑΓΑΘΟΝΗΣΙΟΥ   
15    2022                                              ΑΓΙΑΣ   
16    2022                                     ΑΓΙΑΣ ΒΑΡΒΑΡΑΣ   
17    2022                                   ΑΓΙΑΣ ΠΑΡΑΣΚΕΥΗΣ   
18    2022                                    ΑΓΙΟΥ ΒΑΣΙΛΕΙΟΥ   
19    2022                                    ΑΓΙΟΥ ΔΗΜΗΤΡΙΟΥ   
20    2022                                   ΑΓΙΟΥ ΕΥΣΤΡΑΤΙΟΥ   
21    2022                          ΑΓΙΩΝ ΑΝΑΡΓΥΡΩΝ-ΚΑΜΑΤΕΡΟΥ   
22    2022                                         ΑΓΚΙΣΤΡΙΟΥ   
23    2022                                            ΑΓΡΑΦΩΝ   
24    2022                                           ΑΓΡΙΝΙΟΥ   
25    2022                                           ΑΘΗΝΑΙΩΝ   
26    2022                                            ΑΙΓΑΛΕΩ   
27    2022                                         ΑΙΓΙΑΛΕΙΑΣ   
28    2022                                            ΑΙΓΙΝΑΣ   
29    2022                                    ΑΚΤΙΟΥ-ΒΟΝΙΤΣΑΣ   
30    2022                                       ΑΛΕΞΑΝΔΡΕΙΑΣ   
31    2022                                    ΑΛΕΞΑΝΔΡΟΥΠΟΛΗΣ   
32    2022                                  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   
33    2022                                             ΑΛΙΜΟΥ   
34    2022                                            ΑΛΜΥΡΟΥ   
35    2022                                          ΑΛΟΝΝΗΣΟΥ   
36    2022                                            ΑΜΑΡΙΟΥ   
37    2022                                         ΑΜΑΡΟΥΣΙΟΥ   
38    2022                                            ΑΜΟΡΓΟΥ   
39    2022                              ΑΜΠΕΛΟΚΗΠΩΝ-ΜΕΝΕΜΕΝΗΣ   
40    2022                                          ΑΜΥΝΤΑΙΟΥ   
41    2022                                ΑΜΦΙΚΛΕΙΑΣ-ΕΛΑΤΕΙΑΣ   
42    2022                                         ΑΜΦΙΛΟΧΙΑΣ   
43    2022                                          ΑΜΦΙΠΟΛΗΣ   
44    2022                                   ΑΝΑΤΟΛΙΚΗΣ ΜΑΝΗΣ   
45    2022                                             ΑΝΑΦΗΣ   
46    2022                                ΑΝΔΡΑΒΙΔΑΣ-ΚΥΛΛΗΝΗΣ   
47    2022                             ΑΝΔΡΙΤΣΑΙΝΑΣ-ΚΡΕΣΤΕΝΩΝ   
48    2022                                             ΑΝΔΡΟΥ   
49    2022                                          ΑΝΤΙΠΑΡΟΥ   
50    2022                                         ΑΠΟΚΟΡΩΝΟΥ   
51    2022                                           ΑΡΓΙΘΕΑΣ   
52    2022                                     ΑΡΓΟΥΣ-ΜΥΚΗΝΩΝ   
53    2022                                         ΑΡΙΣΤΟΤΕΛΗ   
54    2022                                           ΑΡΡΙΑΝΩΝ   
55    2022                                            ΑΡΤΑΙΩΝ   
56    2022                                ΑΡΧΑΝΩΝ-ΑΣΤΕΡΟΥΣΙΩΝ   
57    2022                                        ΑΣΠΡΟΠΥΡΓΟΥ   
58    2022                                        ΑΣΤΥΠΑΛΑΙΑΣ   
59    2022                          ΒΑΡΗΣ-ΒΟΥΛΑΣ-ΒΟΥΛΙΑΓΜΕΝΗΣ   
60    2022                               

In [28]:
df=df[['year', 'Δήμος','municipality', 'Περιφέρεια','amount',  'Δασαρχείο',
       'Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας',
       'Συμπληρωματική χρηματοδότηση ΥΠΕΣ',
       'Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας', 'Χρηματοδότηση ΕΣΠΑ',
       'Χρηματοδότηση Αντώνης Τρίτσης', 'Άλλη πηγή χρηματοδότησης ονομαστικά',
       'Δαπάνη για αντιπυρρική προστασία - Ισολογισμός']].copy()

In [29]:
df[df['municipality'].str.contains('ΑΛΙΑΡΤ', na=False)]

year              Δήμος       municipality Περιφέρεια   amount  \
32    2022  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
373   2021  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
714   2020           ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1055  2019           ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1388  2018           ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       #N/A  35000.0   

     Δασαρχείο Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας  \
32                                                                   
373                                                                  
714                                                                  
1055                                                                 
1388                                                                 

     Συμπληρωματική χρηματοδότηση ΥΠΕΣ  \
32                                       
373                                      
714                                      
1055                                     
1388                                     

     Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας Χρηματοδότηση ΕΣΠΑ  \
32                                                                 
373                                                                
714                                                                
1055                                                               
1388                                                               

     Χρηματοδότηση Αντώνης Τρίτσης Άλλη πηγή χρηματοδότησης ονομαστικά  \
32                                                                       
373                                                                      
714                                                                      
1055                                                                     
1388                                                                     

     Δαπάνη για αντιπυρρική προστασία - Ισολογισμός  
32                                                   
373                                                  
714                                                  
1055                                                 
1388

# Σύνδεση του κεντρικού dataset με τους κωδικούς uid

In [30]:
#merge df & uid
master = pd.merge(df, uid_df, left_on='municipality', right_on='municipality', how='outer')

In [31]:
master[master['municipality'].str.contains('ΑΛΙΑΡΤ', na=False)]

year              Δήμος       municipality Περιφέρεια   amount  \
156   2022  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
157   2021  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
1704  2020           ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1705  2019           ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1706  2018           ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       #N/A  35000.0   

     Δασαρχείο Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας  \
156                                                                  
157                                                                  
1704                                                                 
1705                                                                 
1706                                                                 

     Συμπληρωματική χρηματοδότηση ΥΠΕΣ  \
156                                      
157                                      
1704                                     
1705                                     
1706                                     

     Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας Χρηματοδότηση ΕΣΠΑ  \
156                                                                
157                                                                
1704                                                               
1705                                                               
1706                                                               

     Χρηματοδότηση Αντώνης Τρίτσης Άλλη πηγή χρηματοδότησης ονομαστικά  \
156                                                                      
157                                                                      
1704                                                                     
1705                                                                     
1706                                                                     

     Δαπάνη για αντιπυρρική προστασία - Ισολογισμός     uid same uid as  
156                                                  6020.0              
157                                                  6020.0              
1704                                                 6020.0              
1705                                                 6020.0              
1706                                                 6020.0

In [32]:
master.head(1)

year             Δήμος      municipality Περιφέρεια   amount Δασαρχείο  \
0  2022  ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ  ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ     ΗΛΕΙΑΣ  85800.0             

  Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας  \
0                                                       

  Συμπληρωματική χρηματοδότηση ΥΠΕΣ Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας  \
0                            140000                                            

  Χρηματοδότηση ΕΣΠΑ Χρηματοδότηση Αντώνης Τρίτσης  \
0                                                    

  Άλλη πηγή χρηματοδότησης ονομαστικά  \
0                                       

  Δαπάνη για αντιπυρρική προστασία - Ισολογισμός     uid same uid as  
0                                                 6046.0

# Δαπάνες δήμων από Διαύγεια

In [33]:
pay = pd.read_csv(payments, skipinitialspace = True, na_filter=False)
pay = pay.apply(strip_rows)

In [34]:
pay.rename(columns={'publish_year':'year', 'label':'Δήμος','cleaned':'municipality'},inplace=True)

In [35]:
pay = normalize_municipality_names(pay, 2020, 'ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ', 'ΑΛΙΑΡΤΟΥ', 'municipality')

In [36]:
pay[pay['municipality'].str.contains('ΑΛΙΑΡΤ', na=False)]

uid     Δήμος       municipality  year sum_anathesi sum_analipsi  \
21    6020  ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  2022     12439.68       6451.1   
362   6020  ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  2021       464.38          465   
703   6020  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ  2020       4621.4          500   
1044  6020  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ  2019      1548.76          500   
1385  6020  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ  2018                             

     sum_oristikopisi sum_egkrisi  
21            6450.48              
362            464.36              
703               920              
1044          1083.65              
1385

# Ενσωμάτωση δαπανών στο κεντρικό dataset

In [37]:
df = pd.merge(master, pay, left_on=['uid','year'], right_on=['uid','year'], how = 'outer', suffixes=('_master', '_pay'))

In [38]:
df

year                                       Δήμος_master  \
0     2022                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
1     2021                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
2     2020                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
3     2019                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
4     2018                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
5     2022                                            ΑΧΑΡΝΩΝ   
6     2021                                            ΑΧΑΡΝΩΝ   
7     2020                                            ΑΧΑΡΝΩΝ   
8     2019                                            ΑΧΑΡΝΩΝ   
9     2018                                            ΑΧΑΡΝΩΝ   
10    2022                                           ΓΛΥΦΑΔΑΣ   
11    2021                                           ΓΛΥΦΑΔΑΣ   
12    2020                                           ΓΛΥΦΑΔΑΣ   
13    2019                                           ΓΛΥΦΑΔΑΣ   
14    2018                                           ΓΛΥΦΑΔΑΣ   
15    2022                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
16    2021                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
17    2020                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
18    2019                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
19    2018                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
20    2022                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
21    2021                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
22    2020                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
23    2019                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
24    2018                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
25    2022                                          ΜΥΤΙΛΗΝΗΣ   
26    2022                                          ΜΥΤΙΛΗΝΗΣ   
27    2021                                          ΜΥΤΙΛΗΝΗΣ   
28    2020                                          ΜΥΤΙΛΗΝΗΣ   
29    2019                                          ΜΥΤΙΛΗΝΗΣ   
30    2018                                          ΜΥΤΙΛΗΝΗΣ   
31    2022                                           ΠΕΝΤΕΛΗΣ   
32    2021                                           ΠΕΝΤΕΛΗΣ   
33    2020                                           ΠΕΝΤΕΛΗΣ   
34    2019                                           ΠΕΝΤΕΛΗΣ   
35    2018                                           ΠΕΝΤΕΛΗΣ   
36    2022                                              ΡΟΔΟΥ   
37    2021                                              ΡΟΔΟΥ   
38    2020                                              ΡΟΔΟΥ   
39    2019                                              ΡΟΔΟΥ   
40    2018                                              ΡΟΔΟΥ   
41    2022                                           ΣΟΥΦΛΙΟΥ   
42    2021                                           ΣΟΥΦΛΙΟΥ   
43    2020                                           ΣΟΥΦΛΙΟΥ   
44    2019                                           ΣΟΥΦΛΙΟΥ   
45    2018                                           ΣΟΥΦΛΙΟΥ   
46    2022                                             ΩΡΩΠΟΥ   
47    2021                                             ΩΡΩΠΟΥ   
48    2020                                             ΩΡΩΠΟΥ   
49    2019                                             ΩΡΩΠΟΥ   
50    2018                                             ΩΡΩΠΟΥ   
51    2022                                           ΑΛΜΩΠΙΑΣ   
52    2021                                           ΑΛΜΩΠΙΑΣ   
53    2020                                           ΑΛΜΩΠΙΑΣ   
54    2019                                           ΑΛΜΩΠΙΑΣ   
55    2018                                           ΑΛΜΩΠΙΑΣ   
56    2022                                           ΑΝΩΓΕΙΩΝ   
57    2021                                           ΑΝΩΓΕΙΩΝ   
58    2020                                           ΑΝΩΓΕΙΩΝ   
59    2019                                           ΑΝΩΓΕΙΩΝ   
60    2018                               

In [39]:
df[df.municipality_pay.str.contains('ΑΛΙΑΡΤΟΥ',na=False)]

year       Δήμος_master municipality_master Περιφέρεια   amount  \
156   2022  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
157   2021  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
1704  2020           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1705  2019           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1706  2018           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   

     Δασαρχείο Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας  \
156                                                                  
157                                                                  
1704                                                                 
1705                                                                 
1706                                                                 

     Συμπληρωματική χρηματοδότηση ΥΠΕΣ  \
156                                      
157                                      
1704                                     
1705                                     
1706                                     

     Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας Χρηματοδότηση ΕΣΠΑ  ...  \
156                                                               ...   
157                                                               ...   
1704                                                              ...   
1705                                                              ...   
1706                                                              ...   

     Άλλη πηγή χρηματοδότησης ονομαστικά  \
156                                        
157                                        
1704                                       
1705                                       
1706                                       

     Δαπάνη για αντιπυρρική προστασία - Ισολογισμός     uid  same uid as  \
156                                                  6020.0                
157                                                  6020.0                
1704                                                 6020.0                
1705                                                 6020.0                
1706                                                 6020.0                

     Δήμος_pay   municipality_pay sum_anathesi sum_analipsi sum_oristikopisi  \
156   ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ     12439.68       6451.1          6450.48   
157   ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ       464.38          465           464.36   
1704  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       4621.4          500              920   
1705  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ      1548.76          500          1083.65   
1706  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ                                              

     sum_egkrisi  
156               
157               
1704              
1705              
1706              

[5 rows x 21 columns]

# Πυρκαγιές

In [40]:
fr = pd.read_csv(fires)
fr = fr.apply(strip_rows)

In [41]:
fr.head()

dimos  year  incidents_count  Δάση  Δασική Έκταση  Άλση  \
0  ΑΒΔΗΡΩΝ  2018               59   0.0           0.00   0.0   
1  ΑΒΔΗΡΩΝ  2019               73   0.0         190.00   0.0   
2  ΑΒΔΗΡΩΝ  2020               52   0.0           3.03   0.0   
3  ΑΒΔΗΡΩΝ  2021               57   0.0          80.00   0.0   
4  ΑΒΔΗΡΩΝ  2022               62   1.0           2.00   0.0   

   Χορτ/κές Εκτάσεις  Καλάμια - Βάλτοι  Γεωργικές Εκτάσεις  \
0             387.89            180.68               57.20   
1              26.61             12.45              112.35   
2              24.75            172.20               16.20   
3             149.39             15.10                0.00   
4             380.77            233.50              191.20   

   Υπολλείματα Καλλιεργειών  
0                    140.05  
1                    100.37  
2                     77.65  
3                    126.00  
4                     44.30

In [42]:
dimos_replacements = {
    'ΒΟΪΟΥ':'ΒΟΙΟΥ',
    'ΔΥΤΙΚΗΣ ΑΧΑΪΑΣ':'ΔΥΤΙΚΗΣ ΑΧΑΙΑΣ',
    'ΧΑΪΔΑΡΙΟΥ':'ΧΑΙΔΑΡΙΟΥ',
    'ΛΟΥΤΡΑΚΙΟΥ-ΑΓΙΩΝ ΘΕΟΔΩΡΩ':'ΛΟΥΤΡΑΚΙΟΥ-ΠΕΡΑΧΩΡΑΣ-ΑΓΙΩΝ ΘΕΟΔΩΡΩΝ',
    'ΤΡΟΙΖΗΝΙΑΣ':'ΤΡΟΙΖΗΝΙΑΣ-ΜΕΘΑΝΩΝ',
    'ΣΥΡΟΥ-ΕΡΜΟΥΠΟΛΕΩΣ':'ΣΥΡΟΥ-ΕΡΜΟΥΠΟΛΗΣ',
    'ΠΕΤΡΟΥΠΟΛΕΩΣ':'ΠΕΤΡΟΥΠΟΛΗΣ',
    'ΟΡΕΣΤΙΔΟΣ':'ΟΡΕΣΤΙΑΔΑΣ',
#     'ΑΝΑΤΟΛΙΚΗΣ ΣΑΜΟΥ':'ΣΑΜΟΥ',
    'ΜΩΛΟΥ':'ΜΩΛΟΥ-ΑΓ. ΚΩΝΣΤΑΝΤΙΝΟΥ',
    'ΔΩΡΙΔΟΑΣ':'ΔΩΡΙΔΟΣ',
    'ΛΙΜΝΗ ΠΛΑΣΤΗΡΑ':'ΛΙΜΝΗΣ ΠΛΑΣΤΗΡΑ',
    'ΕΥΟΣΜΟΥ-ΚΟΡΔΕΛΙΟΥ':'ΚΟΡΔΕΛΙΟΥ-ΕΥΟΣΜΟΥ'
}

In [43]:
fr.replace(dimos_replacements, inplace=True)

In [44]:
fr = normalize_municipality_names(fr, 2020, 'ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ', 'ΑΛΙΑΡΤΟΥ','dimos')

In [45]:
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

In [46]:
fr[fr.dimos.str.contains('ΑΛΙΑΡΤ')]

dimos  year  incidents_count  Δάση  Δασική Έκταση  Άλση  \
65           ΑΛΙΑΡΤΟΥ  2018                8   2.0            7.0   0.0   
66           ΑΛΙΑΡΤΟΥ  2019               15   0.0            0.0   0.0   
67           ΑΛΙΑΡΤΟΥ  2020               30   0.4           16.7   0.0   
68  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  2021               24   0.0          409.5   0.0   
69  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ  2022               28   1.0          115.5   0.0   

    Χορτ/κές Εκτάσεις  Καλάμια - Βάλτοι  Γεωργικές Εκτάσεις  \
65             150.33              1.00                7.50   
66              25.08             12.10               37.50   
67               5.60              7.40               67.71   
68              50.54              5.27              122.40   
69               5.50             30.25              136.70   

    Υπολλείματα Καλλιεργειών  
65                      0.70  
66                     65.00  
67                     32.56  
68                     45.00  
69                     18.62

In [47]:
fr.shape

(1346, 10)

In [48]:
fr['dimos'].replace(municipalities.values(), municipalities.keys(), inplace=True)

In [49]:
fr_df = pd.merge(df, fr, left_on=['year', 'municipality_master'], right_on=['year','dimos'], how='outer')

In [50]:
fr_df[fr_df.dimos.str.contains('ΑΛΙΑΡΤΟΥ',na=False)]

year       Δήμος_master municipality_master Περιφέρεια   amount  \
156   2022  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
157   2021  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
1709  2020           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1710  2019           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1711  2018           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   

     Δασαρχείο Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας  \
156                                                                  
157                                                                  
1709                                                                 
1710                                                                 
1711                                                                 

     Συμπληρωματική χρηματοδότηση ΥΠΕΣ  \
156                                      
157                                      
1709                                     
1710                                     
1711                                     

     Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας Χρηματοδότηση ΕΣΠΑ  \
156                                                                
157                                                                
1709                                                               
1710                                                               
1711                                                               

     Χρηματοδότηση Αντώνης Τρίτσης Άλλη πηγή χρηματοδότησης ονομαστικά  \
156                                                                      
157                                                                      
1709                                                                     
1710                                                                     
1711                                                                     

     Δαπάνη για αντιπυρρική προστασία - Ισολογισμός     uid same uid as  \
156                                                  6020.0               
157                                                  6020.0               
1709                                                 6020.0               
1710                                                 6020.0               
1711                                                 6020.0               

     Δήμος_pay   municipality_pay sum_anathesi sum_analipsi sum_oristikopisi  \
156   ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ     12439.68       6451.1          6450.48   
157   ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ       464.38          465           464.36   
1709  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       4621.4          500              920   
1710  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ      1548.76          500          1083.65   
1711  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ                                              

     sum_egkrisi              dimos  incidents_count  Δάση  Δασική Έκταση  \
156               ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ             28.0   1.0          115.5   
157               ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ             24.0   0.0          409.5   
1709                       ΑΛΙΑΡΤΟΥ             30.0   0.4           16.7   
1710                       ΑΛΙΑΡΤΟΥ             15.0   0.0            0.0   
1711                       ΑΛΙΑΡΤΟΥ              8.0   2.0            7.0   

      Άλση  Χορτ/κές Εκτάσεις  Καλάμια - Βάλτοι  Γεωργικές Εκτάσεις  \
156    0.0               5.50             30.25              136.70   
157    0.0              50.54              5.27              122.40   
1709   0.0               5.60              7.40               67.71   
1710   0.0              25.08             12.10               37.50   
1711   0.0             150.33              1.00                7.50   

      Υπολλείματα Καλλιεργειών  
156                      18.62  
157                      45.00  
1709                     32.56  
1710                     65.00  
1711                      0.70

In [51]:
fr_df

year                                       Δήμος_master  \
0     2022                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
1     2021                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
2     2020                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
3     2019                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
4     2018                                   ΑΡΧΑΙΑΣ ΟΛΥΜΠΙΑΣ   
5     2022                                            ΑΧΑΡΝΩΝ   
6     2021                                            ΑΧΑΡΝΩΝ   
7     2020                                            ΑΧΑΡΝΩΝ   
8     2019                                            ΑΧΑΡΝΩΝ   
9     2018                                            ΑΧΑΡΝΩΝ   
10    2022                                           ΓΛΥΦΑΔΑΣ   
11    2021                                           ΓΛΥΦΑΔΑΣ   
12    2020                                           ΓΛΥΦΑΔΑΣ   
13    2019                                           ΓΛΥΦΑΔΑΣ   
14    2018                                           ΓΛΥΦΑΔΑΣ   
15    2022                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
16    2021                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
17    2020                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
18    2019                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
19    2018                                   ΙΣΤΙΑΙΑΣ-ΑΙΔΗΨΟΥ   
20    2022                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
21    2021                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
22    2020                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
23    2019                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
24    2018                      ΜΑΝΤΟΥΔΙΟΥ-ΛΙΜΝΗΣ-ΑΓΙΑΣ ΑΝΝΑΣ   
25    2022                                          ΜΥΤΙΛΗΝΗΣ   
26    2022                                          ΜΥΤΙΛΗΝΗΣ   
27    2021                                          ΜΥΤΙΛΗΝΗΣ   
28    2020                                          ΜΥΤΙΛΗΝΗΣ   
29    2019                                          ΜΥΤΙΛΗΝΗΣ   
30    2018                                          ΜΥΤΙΛΗΝΗΣ   
31    2022                                           ΠΕΝΤΕΛΗΣ   
32    2021                                           ΠΕΝΤΕΛΗΣ   
33    2020                                           ΠΕΝΤΕΛΗΣ   
34    2019                                           ΠΕΝΤΕΛΗΣ   
35    2018                                           ΠΕΝΤΕΛΗΣ   
36    2022                                              ΡΟΔΟΥ   
37    2021                                              ΡΟΔΟΥ   
38    2020                                              ΡΟΔΟΥ   
39    2019                                              ΡΟΔΟΥ   
40    2018                                              ΡΟΔΟΥ   
41    2022                                           ΣΟΥΦΛΙΟΥ   
42    2021                                           ΣΟΥΦΛΙΟΥ   
43    2020                                           ΣΟΥΦΛΙΟΥ   
44    2019                                           ΣΟΥΦΛΙΟΥ   
45    2018                                           ΣΟΥΦΛΙΟΥ   
46    2022                                             ΩΡΩΠΟΥ   
47    2021                                             ΩΡΩΠΟΥ   
48    2020                                             ΩΡΩΠΟΥ   
49    2019                                             ΩΡΩΠΟΥ   
50    2018                                             ΩΡΩΠΟΥ   
51    2022                                           ΑΛΜΩΠΙΑΣ   
52    2021                                           ΑΛΜΩΠΙΑΣ   
53    2020                                           ΑΛΜΩΠΙΑΣ   
54    2019                                           ΑΛΜΩΠΙΑΣ   
55    2018                                           ΑΛΜΩΠΙΑΣ   
56    2022                                           ΑΝΩΓΕΙΩΝ   
57    2021                                           ΑΝΩΓΕΙΩΝ   
58    2020                                           ΑΝΩΓΕΙΩΝ   
59    2019                                           ΑΝΩΓΕΙΩΝ   
60    2018                               

In [61]:
fr_df[fr_df['municipality_pay'].str.contains('ΚΑΜΜΕΝ', na=False)]

Empty DataFrame
Columns: [year, Δήμος_master, municipality_master, Περιφέρεια, amount, Δασαρχείο, Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας, Συμπληρωματική χρηματοδότηση ΥΠΕΣ, Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας, Χρηματοδότηση ΕΣΠΑ, Χρηματοδότηση Αντώνης Τρίτσης, Άλλη πηγή χρηματοδότησης ονομαστικά, Δαπάνη για αντιπυρρική προστασία - Ισολογισμός, uid, same uid as, Δήμος_pay, municipality_pay, sum_anathesi, sum_analipsi, sum_oristikopisi, sum_egkrisi, dimos, incidents_count, Δάση, Δασική Έκταση, Άλση, Χορτ/κές Εκτάσεις, Καλάμια - Βάλτοι, Γεωργικές Εκτάσεις, Υπολλείματα Καλλιεργειών]
Index: []

In [53]:
# petame to agio oros

## δοκιμαστικός κώδικας

In [54]:
aliart=fr_df[fr_df.dimos.str.contains('ΑΛΙΑΡΤΟΥ', na=False)]
aliart

year       Δήμος_master municipality_master Περιφέρεια   amount  \
156   2022  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
157   2021  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ   ΒΟΙΩΤΙΑΣ  35000.0   
1709  2020           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1710  2019           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   
1711  2018           ΑΛΙΑΡΤΟΥ            ΑΛΙΑΡΤΟΥ       #N/A  35000.0   

     Δασαρχείο Κατάθεση αντιπυρκού σχεδίου στη ΓΓ. Πολ. Προστασίας  \
156                                                                  
157                                                                  
1709                                                                 
1710                                                                 
1711                                                                 

     Συμπληρωματική χρηματοδότηση ΥΠΕΣ  \
156                                      
157                                      
1709                                     
1710                                     
1711                                     

     Πρόταση χρηματοδότησης ΓΓ Πολ Προστασίας Χρηματοδότηση ΕΣΠΑ  \
156                                                                
157                                                                
1709                                                               
1710                                                               
1711                                                               

     Χρηματοδότηση Αντώνης Τρίτσης Άλλη πηγή χρηματοδότησης ονομαστικά  \
156                                                                      
157                                                                      
1709                                                                     
1710                                                                     
1711                                                                     

     Δαπάνη για αντιπυρρική προστασία - Ισολογισμός     uid same uid as  \
156                                                  6020.0               
157                                                  6020.0               
1709                                                 6020.0               
1710                                                 6020.0               
1711                                                 6020.0               

     Δήμος_pay   municipality_pay sum_anathesi sum_analipsi sum_oristikopisi  \
156   ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ     12439.68       6451.1          6450.48   
157   ΑΛΙΑΡΤΟΥ  ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ       464.38          465           464.36   
1709  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ       4621.4          500              920   
1710  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ      1548.76          500          1083.65   
1711  ΑΛΙΑΡΤΟΥ           ΑΛΙΑΡΤΟΥ                                              

     sum_egkrisi              dimos  incidents_count  Δάση  Δασική Έκταση  \
156               ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ             28.0   1.0          115.5   
157               ΑΛΙΑΡΤΟΥ-ΘΕΣΠΙΕΩΝ             24.0   0.0          409.5   
1709                       ΑΛΙΑΡΤΟΥ             30.0   0.4           16.7   
1710                       ΑΛΙΑΡΤΟΥ             15.0   0.0            0.0   
1711                       ΑΛΙΑΡΤΟΥ              8.0   2.0            7.0   

      Άλση  Χορτ/κές Εκτάσεις  Καλάμια - Βάλτοι  Γεωργικές Εκτάσεις  \
156    0.0               5.50             30.25              136.70   
157    0.0              50.54              5.27              122.40   
1709   0.0               5.60              7.40               67.71   
1710   0.0              25.08             12.10               37.50   
1711   0.0             150.33              1.00                7.50   

      Υπολλείματα Καλλιεργειών  
156                      18.62  
157                      45.00  
1709                     32.56  
1710                     65.00  
1711                      0.70

In [55]:
def isEnglish(s):
  return s.isascii()

print(isEnglish("Test"))
print(isEnglish("ΑΛΙΑΡΤΟΥ"))



True
False


In [56]:
aliar=fr_df[fr_df.municipality_x.str.contains('ΑΛΙΑΡΤ', na=False)]
aliar

AttributeError: 'DataFrame' object has no attribute 'municipality_x'

In [ ]:
print(aliar)

In [ ]:
master[master.municipality=='ΝΕΑΣ ΦΙΛΑΔΕΛΦΕΙΑΣ-ΝΕΑΣ ΧΑΛΚΗΔΟΝΑΣ']

In [ ]:
master[master['municipality'].str.contains('ΦΙΛΑΔΕΛΦΕΙΑΣ', na=False)]

In [ ]:
print

In [ ]:
lst = master_df.municipality.unique().tolist()

In [ ]:
for item in lst:
    value = difflib.get_close_matches(item, lst, n=2, cutoff=0.70)
    if len(value)>1:
        print(f"{item} -> {value}")

In [ ]:
df.columns

In [ ]:
# a = df[['year', 'Δήμος', 'amount']].copy()